In [31]:
!pip install spacy sklearn rowordnet
!python -m spacy download ro_core_news_lg
!pip install bs4
!pip install sparknlp pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=5a9fea0fe96758fcc6537333c34eea18231dd9758cb4144f66d018929b90b873
  Stored in directory: /root/.cache/pip/wheels/14/25/f7/1cc0956978ae479e75140219088deb7a36f60459df242b1a72
Successfully built sklearn
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-01-20 08:48:27.293917: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.5/568.5 MB 1.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ro

In [65]:
from bs4 import BeautifulSoup
import spacy

nlp = spacy.load("ro_core_news_lg")


def parse_xml(path):
  with open(path, encoding='utf8') as fp:
    file_p = fp.read()
    soup = BeautifulSoup(file_p, 'html.parser')
  data = soup.find_all('instance')
  parsed_data = {}
  for sense in data:
    new_data = []
    contexts = sense.find_all('context')
    senseId = sense.attrs["id"][10:]
    if len(senseId)<=1:
      break
    word = ''
    for context in contexts:
      try:
        word = nlp(context.head.get_text())[0].lemma_
        sentence = context.decode_contents().strip('\n')
      
        
        #print(word)
        a, *b = sentence.split('<head>')
        b, *c = ''.join(b).split('</head>')
        sentence = a + b + ''.join(c)
        sentence= ' '.join(sentence.split())
        new_data.append((sentence, senseId))
          
      except:
        continue
    if word =='':
      continue
    if word in parsed_data:
      parsed_data[word].extend(new_data)
    else:
      parsed_data[word] = new_data
     
  return parsed_data



prep_data = parse_xml("/content/drive/MyDrive/wsd/data_wsd/adnotari_Calina.xml")
print(prep_data.keys())
print(prep_data["lac"][0])



dict_keys(['lac', 'lin', 'mare', 'masă'])
('din 18 ianuarie 2014 Toate Articolele Autorului iubirea îi e ochi de curcubeu desprinsă din mângâierea părinților deasupra unui leagăn cu povești citea pe o scară de lemn la cinci ani despre lebede ce mergeau printre trestii la culcare pe lac punea bărcuțe pe pârâu și odată l-a întrebat o fetiță unde îi e tatăl i-a răspuns că e după gratii lângă al lui și lângă bunicu-său i-a fost de ajuns copil fiind să ucidă cu praștia o', 'ENG30-09328904-n')


In [62]:
import random
def split_data(data):
  train_data = {}
  test_data = {}
  for word, vector in data.items():
    l = len(vector)
    marg = 2*l//3
    random.shuffle(vector)
    train_data[word] = vector[:marg]
    test_data[word] = vector[marg:]
  return train_data, test_data

train_data, test_data = split_data(prep_data)
print(train_data["masă"])
print(test_data["lac"])

[('ea. De ce nu-i ajunsese numai iubirea ei curată? Mașinal băgă cheia în iala ușii lor. Locuia cu părinții și cu un frate mai mare. Se îndreptă spre camera ei, fără să bage de seamă, că în bucătărie, mama pregătea masa. Se aruncă pe pat și închise ochii. Revăzu în minte toată scena, în cele mai mici detalii. Simți aproape fizic, gesturile lui Maur. O mână caldă i se așeză pe frunte. Tresări ca scuturată de friguri. Era mama, care foarte', 'ENG30-07573696-n'), ('- Și eu de ce numai țin minte ? - Calul s-a speriat și ați căzut, trei zile nu ați știut de domnia voastră. Brusc parcă simți retrăind trupul aflat în cădere urmată de senzația de moliciune. - Da îmi este foame, voi lua masa aici ! - Am înțeles măria ta ! deândată ... și se făcu nevăzut. Își întinse brațele și luă aer în pieptul ce se umflă a împlinire. Se scufundă apoi în fotoliul din fața geamului deschis și se născu în mintea ei întrebarea - eu ce', 'ENG30-07573696-n'), ('Acasa &gt; Poezie &gt; Oglindire &gt; RULETA RUSEASCĂ

In [66]:
import nltk
import spacy
from sklearn.feature_extraction import DictVectorizer
import rowordnet as rwn
from nltk.classify import MaxentClassifier

vec = DictVectorizer()
wn = rwn.RoWordNet()
nlp = spacy.load("ro_core_news_lg")


def extract_hypernymes(word):
    hyper = []
    synset_ids = wn.synsets(literal=word.lemma_, pos=rwn.synset.Synset.Pos.NOUN)
    if len(synset_ids) > 0:
        c = wn.synset_to_hypernym_root(synset_ids[0])
        for c1 in c:
            if len(wn(c1).literals):
                hyper.append(wn(c1).literals[0])
    hyper_dict = {}
    for i in range(len(hyper)):
      hyper_dict["hyper"+str(i)] = hyper[i]
    #print(hyper_dict)
    return hyper_dict


def extract_feat_for_word(word, rel):
    hyper = {}
    if word.pos_ == "NOUN":
        hyper = extract_hypernymes(word)
    d = {
        ("text"+rel): word.text,
        "pos"+rel: word.pos_,
        "morpho"+rel: word.morph,
        "lemma"+rel: word.lemma_,
        "dep"+rel: word.dep_,
         "ner"+rel: word.ent_type_,
        #"hyper"+rel: hyper,
        #"position_rel": rel
    }
    d.update(hyper)
    return d

def create_features_vector(word, context):
    text = nlp(context)
    vect = {}
    for w in text:
      if w.lemma_ == word:
            if w.head != w:
              
              #print(w.head)
              vect.update(extract_feat_for_word(w.head, "head"))
              for ind, siblings in enumerate(w.head.children):
                vect.update(extract_feat_for_word(siblings, "0_"+str(ind) if w != siblings else "main"))
            else:
              vect.update(extract_feat_for_word(w, "main"))
            for chil in w.children:
              vect.update(extract_feat_for_word(chil, "children"))
        
    return vect

def prepare_data(data):
  new_data_per_words = {}

  for word, instances in data.items():
    sense = word
    #print(sense)
    new_data = []
    #print(instances)
    
    for i in instances:
      #print(i)
      #print(i[0])
      new_data.append((create_features_vector(sense,i[0]), i[1]))
    new_data_per_words[word] = new_data


  return new_data_per_words

a = prepare_data(train_data)

#test_data, test_labels = prepare_test_data(test_se3)
b = prepare_data(test_data)
print(a["lac"])
print(b["lac"])
print(a.keys())

[({'texthead': 'lac', 'poshead': 'NOUN', 'morphohead': Definite=Ind|Gender=Masc|Number=Sing, 'lemmahead': 'lac', 'dephead': 'conj', 'nerhead': '', 'text0_0': ',', 'pos0_0': 'PUNCT', 'morpho0_0': , 'lemma0_0': ',', 'dep0_0': 'punct', 'ner0_0': '', 'text0_1': 'unghii', 'pos0_1': 'NOUN', 'morpho0_1': Definite=Ind|Gender=Fem|Number=Plur, 'lemma0_1': 'unghie', 'dep0_1': 'nmod', 'ner0_1': '', 'hyper0': 'tampon', 'hyper1': 'obturator', 'hyper2': 'dop', 'hyper3': 'obstacol', 'hyper4': 'construcție', 'text0_2': 'Aut', 'pos0_2': 'PROPN', 'morpho0_2': , 'lemma0_2': 'Aut', 'dep0_2': 'nmod', 'ner0_2': 'ORGANIZATION', 'text0_3': 'luciu', 'pos0_3': 'ADJ', 'morpho0_3': Definite=Ind|Degree=Pos|Gender=Masc|Number=Sing, 'lemma0_3': 'luciu', 'dep0_3': 'amod', 'ner0_3': '', 'text0_4': 'fard', 'pos0_4': 'NOUN', 'morpho0_4': Definite=Ind|Gender=Masc|Number=Sing, 'lemma0_4': 'fard', 'dep0_4': 'conj', 'ner0_4': '', 'hyper5': 'artifact', 'hyper6': 'ansamblu', 'text0_5': '...', 'pos0_5': 'PUNCT', 'morpho0_5': , 

In [64]:


algorithm = nltk.classify.MaxentClassifier.ALGORITHMS[0]
classifiers = {}
for w in a.keys():
  print(w)
  classifier = nltk.MaxentClassifier.train(a[w], algorithm, max_iter=20)
  #classifier.show_most_informative_features(40, show='all')
  #print(classifier.weights())
  #print(feat)
  classifiers[w] = classifier

for w in b.keys():
  #output_labels = classifiers[w].classify_many([x[0] for x in b[w]])

  print(nltk.classify.accuracy( classifiers[w], b[w]))

lac
  ==> Training (20 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -1.38629        0.231
             2          -1.12613        0.862
             3          -0.94541        0.877
             4          -0.81578        0.908
             5          -0.71867        0.938
             6          -0.64311        0.969
             7          -0.58248        0.969
             8          -0.53262        0.969
             9          -0.49080        0.969
            10          -0.45515        0.985
            11          -0.42436        0.985
            12          -0.39747        0.985
            13          -0.37377        0.985
            14          -0.35271        0.985
            15          -0.33386        0.985
            16          -0.31689        1.000
            17          -0.30153        1.000
            18          -0.28755        1.000
            19          -0.27478        1.00

In [45]:

import rowordnet as rwn
wn = rwn.RoWordNet()
a = wn('ENG30-00982602-a').literals
print(a)

['domol', 'lin', 'ușor']


In [59]:
import spacy
nlp = spacy.load("ro_core_news_lg")
title = nlp('mării')
print(title[0].lemma_)


mare
